In [2]:
import argparse
import pandas as pd
import file_management
import videoProcessing
import detectMD
import parseResults
import splitData
import predictSpecies

def main(image_dir, model_file, class_model, class_list):
    """
    This function is the main method to invoke all the sub functions
    to create a working directory for the image directory.

    Args:
        image_dir (str): The directory path containing the images or videos.
        model_file (str): The file path of the MegaDetector model.
        class_model (str): The file path of the classifier model.
        class_list (list): A list of classes or species for classification.

    Returns:
        pandas.DataFrame: Concatenated dataframe of animal and empty detections.
    """
    print("Setting up working directory...")
    # Create a working directory, build the file manifest from img_dir
    working_dir = file_management.WorkingDirectory(image_dir)
    files = file_management.build_file_manifest(
        image_dir, out_file=working_dir.filemanifest
        )
    print("Processing videos...")
    # Video-processing to extract individual frames as images in to directory
    all_frames = videoProcessing.images_from_videos(
        files, out_dir=working_dir.vidfdir,
        out_file=working_dir.imageframes, parallel=True, frames=2
        )
    print("Running images and video frames through MegaDetector...")
    # Run all images and video frames through MegaDetector
    md_results = detectMD.detect_MD_batch(
        model_file, all_frames["Frame"],
        checkpoint_path=None, checkpoint_frequency=-1,
        results=None, n_cores=1, quiet=True
        )
    print("Converting MD JSON to pd dataframe and merging with manifest...")
    # Convert MD JSON to pandas dataframe, merge with manifest
    md_res = parseResults.parseMD(
        md_results, manifest=all_frames, out_file=working_dir.mdresults
        )
    print("Extracting animal detections...")
    # Extract animal detections from the rest
    animals = splitData.getAnimals(md_res)
    empty = splitData.getEmpty(md_res)
    print("Predicting species of animal detections...")
    # Use the classifier model to predict the species of animal detections
    pred_results = predictSpecies.predictSpecies(animals, class_model, batch=4)
    print("Applying predictions to animal detections...")
    animals = parseResults.applyPredictions(
        animals, pred_results, class_list, out_file=working_dir.predictions
        )
    print("Concatenating animal and empty dataframes...")
    manifest = pd.concat([animals, empty])
    manifest.to_csv(working_dir.results)
    print("Final Results in " + working_dir.results)

ModuleNotFoundError: No module named 'file_management'

In [3]:
%pwd

'/home/kyra/animl-py/notebooks'

In [3]:
main('/home/kyra/animl-py/examples/Southwest','/home/kyra/animl-py/models/md_v5a.0.0.pt',
    '/home/kyra/animl-py/models/southwest_v2.h5','/home/kyra/animl-py/models/southwest_classes.txt')

Setting up working directory...


NameError: name 'file_management' is not defined